In [10]:
import os
import pandas as pd
import re
import pickle

#Preprocesamiento
import spacy
from spacy.lang.es.stop_words import STOP_WORDS #importar set de stopwords
from nltk.stem import SnowballStemmer #importar stemmer
nlp = spacy.load('es_core_news_sm') #python -m spacy download es

#Topic modeling
import gensim
from gensim.corpora import Dictionary, MmCorpus
from gensim.models.ldamodel import LdaModel
from gensim.models import CoherenceModel
import pyLDAvis
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
from IPython.core.display import display, HTML

#Elimina los warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
def tokenizer(doc, sep=None, vocabulary = None, homol_dict=None, lemmatization=False, stemming = False):
    '''
    Por defecto divide la sentencia por el carácter espacio.
    Ej: 'Data Mining is the best course'->['Data',  'Mining', 'is', 'the', 'best', 'course']
    
    Input: 
    1. doc: str, documento.
    2. sep: str, carácter para dividir el documento en tokens, por defecto es el espacio.
    3. vocabuary: set, si un vocabulario es dado filtra las palabras que no estan presentes en el.
    4. homol_dict: dict, diccionario de homologaciones.
    5. lemmatization: bool, si es True lleva las palabras a su lema.
    6. stemming: bool, si es True lleva las palabas a su raíz.
    
    Output: 
    list, lista de tokens.
    
    Nota: aplicar stemming y lemmatization al mismo tiempo no es correcto.
    '''
    doc = re.sub(r'\S+@\S+', '', doc) #elimina correos electrónicos  
    doc = re.sub(r'[\xa0]', '', doc) #elimina el patrón \xa0
    doc = re.sub(r'[^\w\s]','', doc) #elimina los símbolos de puntuación excepto underscore
    doc = re.sub(r'[_]', '', doc) #elimina underscore
    doc = re.sub(r'[a-zA-Z]+[0-9]+', '', doc) #elimina los tokens que contienen letras y números
    doc = re.sub(r'([ø ÿ þ])', ' ', doc) #reemplazas los símbolos contenidos por un espacio 
    doc = re.sub(r'[0-9]', '', doc) #elimina los tokens númericos
    tokens = doc.split(sep) #tokenización
    tokens = [word.lower() for word in tokens] #pasar todas las palabras a minúsculas
    
    
    if vocabulary is not None:
        #solo considera caracteres que tokens que estan en vocabulary
        tokens = [word for word in tokens if word in vocabulary]
    
    #if homol_dict is not None:
    
    if lemmatization==True:
        tokens = [nlp(word)[0].lemma_ for word in tokens]
        
    if stemming == True:
        stemmer = SnowballStemmer('spanish')
        tokens = [stemmer.stem(word) for word in tokens]
    
    return tokens


In [7]:
#Importar la base de datos
df = pd.read_pickle('../data/robos_prose_v1.pkl')
df.head()

#Importar vocabulario
with open('../data/vocabulary.pickle', 'rb') as f:
    vocabulary = pickle.load(f)

In [15]:
#Corpus tokenizado utilizando el vocabulario
data_tokenized = [tokenizer(doc, vocabulary=vocabulary) for doc in df['sin_relato']]

#Creamos el diccionario a partir de los textos procesados en el formato que necesita LDA en gensim
dictionary = Dictionary(data_tokenized)

#Transformamos el corpus al formato que requiere la librería
#El corpus contiene una representacion numerica de los textos, un texto es representado por una lista de tuplas
#donde el primer elemento de la tupla es la id de la palabra y el segundo es su frecuencia de aparición en el texto.
corpus = [dictionary.doc2bow(text) for text in data_tokenized]

#Guardamos el diccionario y el corpus
dictionary.save('../data/dictionary.dict')
MmCorpus.serialize('../data/corpus.mm', corpus)

In [21]:
lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=5, alpha='auto', eta='auto', random_state=0) 

In [22]:
#guardamos el modelo
lda_model.save('lda_model.model') 

In [23]:
#printear las topn palabras más probables de un tópico
lda_model.show_topic(topicid=1, topn=15)

[('mecanismo', 0.06720624),
 ('estacionado', 0.04111982),
 ('estacionamiento', 0.038681664),
 ('seguridad', 0.03460389),
 ('informados', 0.014400032),
 ('mall', 0.0138035305),
 ('casa', 0.012337404),
 ('encontraba', 0.010977054),
 ('vidrios', 0.01001468),
 ('ingresada', 0.008947119),
 ('ruido', 0.008045356),
 ('suelo', 0.007937963),
 ('ventana', 0.007869375),
 ('llaves', 0.007554966),
 ('vidrio', 0.00734365)]

In [24]:
lda_vis = pyLDAvis.gensim.prepare(topic_model=lda_model, corpus=corpus, dictionary=dictionary, sort_topics=True, R=30)
pyLDAvis.display(lda_vis, template_type='notebook')